In [ ]:
!pip install tensorflow==2.8
!pip install keras==2.8

### Load

In [ ]:
import os
import numpy as np
from music21 import converter, pitch, interval, instrument, note, stream
from matplotlib import pyplot as plt

from xen.visualise import plotPart, plotSequence
from xen.data import SequenceDataSet

data_dir = "../../../ml_data/mutopia_guitar/"

dataset = SequenceDataSet()
dataset.loadMidiDir(data_dir)

print(f'Loaded {len(dataset.songs)} Songs') 


### Filter

In [ ]:
ticksPerQuarter = 4
dataset.filterFractionalOffsets(ticksPerQuarter)

print(f'{len(dataset.songs)} Songs') 

In [ ]:
sequences = []

ticksPerQuarter = 4   # 4 = 16th notes, to allow triplets would need to be 12 = 48 per measure
measuresPerSequence = 4

sequences, minPitch, maxPitch = dataset.makeSequences(ticksPerQuarter, measuresPerSequence, '4/4')
print(sequences.shape)


trainInput = sequences[0:, 0:, minPitch:maxPitch+1]  # remove unused pitch classes
print(trainInput.shape)

numSequences = trainInput.shape[0]
sequenceLength = trainInput.shape[1]
numCategories = trainInput.shape[2] #Category per midi note

flatTrainInput = trainInput.reshape(numSequences, sequenceLength * numCategories)

print(flatTrainInput.shape)
# # print(flatTrainInput)

# inputShape = (flatTrainInput.shape[1])

# plotSequence(trainInput[1])


In [ ]:
import tensorflow as tf

inputShape = inputShape
latentShape = (2)

# encoder model
encoderInput = tf.keras.layers.Input(shape = inputShape)
encoderOutput = tf.keras.layers.Dense(latentShape, activation = 'tanh')(encoderInput)
encoder = tf.keras.Model(encoderInput, encoderOutput)

# decoder model
decoderInput = tf.keras.layers.Input(shape = latentShape)
decoderOutput = tf.keras.layers.Dense(inputShape, activation = 'sigmoid')(decoderInput)
decoder = tf.keras.Model(decoderInput, decoderOutput)

# autoencoder model
autoencoder = tf.keras.Model(encoderInput, decoder(encoderOutput))

In [ ]:

autoencoder.compile(loss = 'binary_crossentropy', optimizer='rmsprop')

autoencoder.fit(flatTrainInput, flatTrainInput, epochs = 50)


In [ ]:
input = np.array([flatTrainInput[0]])

output = autoencoder.predict(input)

output = output.reshape(1, sequenceLength, numCategories)

print(output.shape)
print(output)